# Noisy Deep Q-Networks

Noisy Networks were introduced to address exploration-exploitation challenges in reinforcement learning. The idea is to __add noise directly to the network's parameters__, allowing the model to explore actions more effectively. The Noisy Deep Q-Network (NDQN) is an extension of the Deep Q-Network (DQN) algorithm, incorporating this concept of parameterized noise for exploration.

__Let's understend the Exploration Challenges in DQN:__
> In vanilla DQN, exploration is typically achieved through __epsilon-greedy strategies__, where the agent randomly explores with a probability epsilon. However, this approach __can be inefficient__, and the agent might __struggle to explore the state-action space thoroughly.__


__That's what Noisy Networks solve:__
> Noisy Networks, introduced by Fortunato et al. in their paper "__Noisy Networks for Exploration__," propose a different approach to exploration. Instead of relying on a separate exploration strategy, __Noisy Networks inject randomness directly into the network's parameters.__


__Noisy DQN Architecture__
- The architecture of a Noisy DQN consists of a neural network that represents the Q-function. The key innovation is the addition of factorized Gaussian noise to the weights of the fully connected layers. The noise is parameterized and adapted during the learning process.
- Consider a fully connected layer with weights W and biases b. In a Noisy Network, each weight $W_i$ and bias $b_i$ is modeled as a factorized Gaussian noise:
  - $W_i = μ_i + σ_i * ϵ_i$
  - $b_i = μ'_i + σ'_i * ϵ'_i$
    - Here:
      - $μ_i$ and $μ'_i$ are the means of the weights and biases, respectively.
      - $σ_i$ and $σ'_i$ are the standard deviations of the weights and biases, respectively.
      - $ϵ_i$ and $ϵ'_i$ are random variables sampled from factorized Gaussian noise with a mean of 0 and a standard deviation of 1.


__Training Noisy DQN:__

> During training, the network learns to __adapt the means and standard deviations of the noisy parameters__. The __stochastic nature of the noisy weights encourages exploration__ because the same input can lead to different actions due to the perturbations in the weights.

The objective during training is to optimize the standard deviations $σ_i$ and $σ'_i$ __to control the amount of noise added to the weights(!)__


The loss function typically includes the standard Q-learning loss (e.g., mean squared error between predicted Q-values and target Q-values) and an additional term penalizing the standard deviations.

In [ ]:
!apt-get install -y xvfb

!pip install pygame
!pip install gym==0.18
!pip install stable-baselines3
!pip install pytorch-lightning==1.6.0
!pip install pyvirtualdisplay

!pip install git+https://github.com/GrupoTuring/PyGame-Learning-Environment
!pip install git+https://github.com/lusob/gym-ple

#### Setup virtual display

In [ ]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

#### Import the necessary code libraries

In [ ]:
import copy
import torch
import random
import gym
import gym_ple
import matplotlib

import numpy as np
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import TransformObservation

from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv, WarpFrame


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [ ]:
# Copied from: https://colab.research.google.com/github/deepmind/dm_control/blob/master/tutorial.ipynb#scrollTo=gKc1FNhKiVJX

def display_video(frames, framerate=30):
  height, width, _ = frames[0].shape
  dpi = 70
  orig_backend = matplotlib.get_backend()
  matplotlib.use('Agg')
  fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
  matplotlib.use(orig_backend)
  ax.set_axis_off()
  ax.set_aspect('equal')
  ax.set_position([0, 0, 1, 1])
  im = ax.imshow(frames[0])
  def update(frame):
    im.set_data(frame)
    return [im]
  interval = 1000/framerate
  anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                  interval=interval, blit=True, repeat=False)
  return HTML(anim.to_html5_video())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Create the Deep Q-Network

In [ ]:
import math
from torch.nn.init import kaiming_uniform_, zeros_

class NoisyLinear(nn.Module):
  """
  A linear layer that generate noise.

  In in NDQN, Noise Layer is a technique that encourage the agent to explore
  more actions in different state.

  One of the advantages if the Noise layer, is that the parameters that
  generate the noise (std) are learnble and updates over the time.
  In other words, the DQN'll learn what the optimal noise to produce for
  find the right balance balance between exploration & exploitation

  - sigma: The bigger the sigma valued, the more noisy the linear layer will be.
  """

  def __init__(self, in_features, out_features, sigma):
    super(NoisyLinear, self).__init__()

    # Initial Learnable parameters for means
    self.w_mu = nn.Parameter(torch.empty((out_features, in_features))) # weights
    self.b_mu = nn.Parameter(torch.empty((out_features))) # bias

    # Initial Learnable parameters for standard deviations (noise)
    self.w_sigma = nn.Parameter(torch.empty((out_features, in_features))) # weights
    self.b_sigma = nn.Parameter(torch.empty((out_features))) # bias

    # Initialize learnble parameters (weights) using Kaiming uniform for means
    kaiming_uniform_(self.w_mu, a=math.sqrt(5))
    zeros_(self.b_mu)

    # Initialize learnble parameters (weights) using Kaiming uniform
    # for standard deviations (noise)
    kaiming_uniform_(self.w_sigma, a=math.sqrt(5))
    zeros_(self.b_sigma)

  def forward(self, x, sigma=0.5):
    """
    Forward method to compute the output of the Noisy Linear layer.

    Parameters:
    -----------
    - x: Input tensor
    - sigma: Standard deviation for the noise (default: 0.5)

    Returns:
    - Output tensor
    """
    if self.training:
      # During training, add noise to weights and biases
      # create normal distribution noises for wights and bias with a given std
      w_noise = torch.normal(0, sigma, size=self.w_mu.size()).to(device)
      b_noise = torch.normal(0, sigma, size=self.b_mu.size()).to(device)

      # Compute the linear transformation with added noise
      return F.linear(
          x, # inpute tensor
          self.w_mu + self.w_sigma * w_noise, # Add the weigths noise
          self.b_mu + self.b_sigma * b_noise # Add the bias noise
        )
    else:
      # During evaluation, no noise is added
      return F.linear(x, self.W_mu, self.b_mu)

In [ ]:
class DQN(nn.Module):
  """
  Deep noisy Q-network.

  Returns:
    - the Q-values ([Q(s,a), Q(s,a), ...])
  """
  def __init__(self, hidden_size, obs_shape, n_actions, sigma=0.5):
    super().__init__()

    # Convolution network
    self.conv = nn.Sequential(
      nn.Conv2d(obs_shape[0], 64, kernel_size=3),
      nn.MaxPool2d(kernel_size=4),
      nn.ReLU(),
      nn.Conv2d(64, 16, kernel_size=3),
      nn.MaxPool2d(kernel_size=4),
      nn.ReLU()
    )

    # Adjust the input shape from conventional layers to linear layers
    # (vectorize the shape to be [f, f, f, ...])
    conv_out_size = self._get_conv_out(obs_shape)

    # Noisy linear layer
    self.head = nn.Sequential(
      NoisyLinear(conv_out_size, hidden_size, sigma=sigma),
      nn.ReLU(),
    )

    # Advantage
    self.fc_adv = NoisyLinear(hidden_size, n_actions, sigma=sigma)

    # Value
    self.fc_value = NoisyLinear(hidden_size, 1, sigma=sigma)


  def _get_conv_out(self, shape):
    """
    Vectorize a < 1D shape into a size vector of 1D.
    """
    conv_out = self.conv(torch.zeros(1, *shape))
    return int(np.prod(conv_out.size()))

  def forward(self, x):

    # Pass Convolution layers, then flatten the output to 1D
    x = self.conv(x.float()).view(x.size()[0], -1)
    # Pass the flatten input to the Noisy layer
    x = self.head(x)

    adv = self.fc_adv(x) # Advantage
    value = self.fc_value(x) # Value

    # Return [Q(s,a),Q(s,a),...]
    return value + adv - torch.mean(adv, dim=1, keepdim=True)

#### Create the policy

In [ ]:
def greedy(state, net):
  """
  A greedy policy. Choose the best action ONLY.

  Why there is no epsilon? beacuse now, we added noise to the DQN,
  so we dont neet to care about balancing exploration & exploitation.
  """
  # Make sure your state is a tensor.
  state = torch.tensor([state]).to(device)
  # Predict the [Q(s,a), Q(s,a), ...]
  # its represent the values of each action that interact in this state.
  q_values = net(state)
  # now, choose the BEST action for this state (the yield the highest Q(s,a))
  _, action = torch.max(q_values, dim=1)
  # Make sure the action is integer
  action = int(action.item())
  # return action.
  return action

#### Create the replay buffer

In [ ]:
class ReplayBuffer:
  """
  A Replay buffer memory that implement the Prioritized Experienc Replay technique.
  """

  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)
    self.priorities = deque(maxlen=capacity)
    self.capacity = capacity
    self.alpha = 0.0  # anneal.
    self.beta = 1.0  # anneal.
    self.max_priority = 0.0

  def __len__(self):
    return len(self.buffer)

  def append(self, experience):
    self.buffer.append(experience)
    self.priorities.append(self.max_priority)

  def update(self, index, priority):
    if priority > self.max_priority:
      self.max_priority = priority
    self.priorities[index] = priority

  def sample(self, batch_size):
    prios = np.array(self.priorities, dtype=np.float64) + 1e-4 # Stability constant.
    prios = prios ** self.alpha
    probs = prios / prios.sum()

    weights = (self.__len__() * probs) ** -self.beta
    weights = weights / weights.max()

    idx = random.choices(range(self.__len__()), weights=probs, k=batch_size)
    sample = [(i, weights[i], *self.buffer[i]) for i in idx]
    return sample

In [ ]:
class RLDataset(IterableDataset):
  """
  Iterable dataset
  """

  def __init__(self, buffer, sample_size=400):
    self.buffer = buffer
    self.sample_size = sample_size

  def __iter__(self):
    for experience in self.buffer.sample(self.sample_size):
      yield experience

#### Create the environment

In [ ]:
class RunningMeanStd:
    # https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Parallel_algorithm
    def __init__(self, epsilon=1e-4, shape=()):
        self.mean = np.zeros(shape, "float64")
        self.var = np.ones(shape, "float64")
        self.count = epsilon

    def update(self, x):
        batch_mean = np.mean(x, axis=0)
        batch_var = np.var(x, axis=0)
        batch_count = x.shape[0]
        self.update_from_moments(batch_mean, batch_var, batch_count)

    def update_from_moments(self, batch_mean, batch_var, batch_count):
        self.mean, self.var, self.count = update_mean_var_count_from_moments(
            self.mean, self.var, self.count, batch_mean, batch_var, batch_count
        )


def update_mean_var_count_from_moments(
    mean, var, count, batch_mean, batch_var, batch_count
):
    delta = batch_mean - mean
    tot_count = count + batch_count

    new_mean = mean + delta * batch_count / tot_count
    m_a = var * count
    m_b = batch_var * batch_count
    M2 = m_a + m_b + np.square(delta) * count * batch_count / tot_count
    new_var = M2 / tot_count
    new_count = tot_count

    return new_mean, new_var, new_count


class NormalizeObservation(gym.core.Wrapper):
    def __init__(
        self,
        env,
        epsilon=1e-8,
    ):
        super().__init__(env)
        self.num_envs = getattr(env, "num_envs", 1)
        self.is_vector_env = getattr(env, "is_vector_env", False)
        if self.is_vector_env:
            self.obs_rms = RunningMeanStd(shape=self.single_observation_space.shape)
        else:
            self.obs_rms = RunningMeanStd(shape=self.observation_space.shape)
        self.epsilon = epsilon

    def step(self, action):
        obs, rews, dones, infos = self.env.step(action)
        if self.is_vector_env:
            obs = self.normalize(obs)
        else:
            obs = self.normalize(np.array([obs]))[0]
        return obs, rews, dones, infos

    def reset(self, **kwargs):
        return_info = kwargs.get("return_info", False)
        if return_info:
            obs, info = self.env.reset(**kwargs)
        else:
            obs = self.env.reset(**kwargs)
        if self.is_vector_env:
            obs = self.normalize(obs)
        else:
            obs = self.normalize(np.array([obs]))[0]
        if not return_info:
            return obs
        else:
            return obs, info

    def normalize(self, obs):
        self.obs_rms.update(obs)
        return (obs - self.obs_rms.mean) / np.sqrt(self.obs_rms.var + self.epsilon)


class NormalizeReward(gym.core.Wrapper):
    def __init__(
        self,
        env,
        gamma=0.99,
        epsilon=1e-8,
    ):
        super().__init__(env)
        self.num_envs = getattr(env, "num_envs", 1)
        self.is_vector_env = getattr(env, "is_vector_env", False)
        self.return_rms = RunningMeanStd(shape=())
        self.returns = np.zeros(self.num_envs)
        self.gamma = gamma
        self.epsilon = epsilon

    def step(self, action):
        obs, rews, dones, infos = self.env.step(action)
        if not self.is_vector_env:
            rews = np.array([rews])
        self.returns = self.returns * self.gamma + rews
        rews = self.normalize(rews)
        self.returns[dones] = 0.0
        if not self.is_vector_env:
            rews = rews[0]
        return obs, rews, dones, infos

    def normalize(self, rews):
        self.return_rms.update(self.returns)
        return rews / np.sqrt(self.return_rms.var + self.epsilon)

In [ ]:
env = gym.make('Catcher-v0')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
obs = env.reset()
obs.shape

In [ ]:
env.observation_space, env.action_space

In [ ]:
for i in range(20):
  obs, rew, done, info = env.step(env.action_space.sample())

plt.imshow(obs)

In [ ]:
env = MaxAndSkipEnv(env, skip=2)

In [ ]:
obs = env.reset()

for i in range(10):
  obs, _, _, _ = env.step(env.action_space.sample())

In [ ]:
type(obs), obs.shape

In [ ]:
plt.imshow(obs)

In [ ]:
env = WarpFrame(env, height=42, width=42)

In [ ]:
obs = env.reset()

for i in range(10):
  obs, _, _, _ = env.step(env.action_space.sample())

In [ ]:
type(obs), obs.shape

In [ ]:
plt.imshow(obs.squeeze(), cmap='gray_r')

In [ ]:
obs.min(), obs.max()

In [ ]:
env = TransformObservation(env, lambda x: x.swapaxes(-1, 0))
env.observation_space = gym.spaces.Box(low=0, high=255, shape=(1, 42, 42), dtype=np.float32)

In [ ]:
obs = env.reset()

for i in range(10):
  obs, _, _, _ = env.step(env.action_space.sample())

In [ ]:
obs.shape

In [ ]:
def create_environment(env_name):
  env = gym_ple.make(env_name)
  env = MaxAndSkipEnv(env, skip=2)
  env = WarpFrame(env, height=42, width=42)
  env = TransformObservation(env, lambda x: x.swapaxes(-1, 0))
  env.observation_space = gym.spaces.Box(low=0, high=255, shape=(1, 42, 42), dtype=np.float32)
  env = NormalizeObservation(env)
  env = NormalizeReward(env)
  return env

#### Create the Deep Q-Learning algorithm

In [4]:
class DeepQLearning(LightningModule):
  """
  Implements the Deep Q Learning with Prioritized Experience Replay
  and with Noisy DQN

  Parameters:
  - env_name (str): The name of the OpenAI Gym environment.
  - policy (callable): The policy function for selecting actions (default: greedy).
  - capacity (int): The capacity of the replay buffer (default: 100,000).
  - batch_size (int): The batch size used for training (default: 256).
  - lr (float): The learning rate for the Q-network optimizer (default: 1e-3).
  - hidden_size (int): The size of the hidden layer in the Q-network (default: 128).
  - gamma (float): The discount factor for future rewards (default: 0.99).
  - loss_fn (callable): The loss function used for training (default: F.smooth_l1_loss).
  - optim (torch.optim.Optimizer): The optimizer for the Q-network (default: AdamW).
  - samples_per_epoch (int): The number of samples collected per epoch (default: 1,000).
  - sync_rate (int): The frequency at which the target Q-network is synchronized with the Q-network (default: 10).
  - sigma (float): The standard deviation parameter for Noisy DQN (default: 0.5).
  - a_start (float): The starting value of alpha (priority exponent) for prioritized experience replay (default: 0.5).
  - a_end (float): The ending value of alpha for prioritized experience replay (default: 0.0).
  - a_last_episode (int): The episode at which alpha reaches its ending value (default: 100).
  - b_start (float): The starting value of beta (importance sampling exponent) for prioritized experience replay (default: 0.4).
  - b_end (float): The ending value of beta for prioritized experience replay (default: 1.0).
  - b_last_episode (int): The episode at which beta reaches its ending value (default: 100).
  """

  # Initialize.
  def __init__(self, env_name, policy=greedy, capacity=100_000,
               batch_size=256, lr=1e-3, hidden_size=128, gamma=0.99,
               loss_fn=F.smooth_l1_loss, optim=AdamW, samples_per_epoch=1_000,
               sync_rate=10, sigma=0.5, a_start=0.5, a_end=0.0, a_last_episode=100,
               b_start=0.4, b_end=1.0, b_last_episode=100):

    super().__init__()

    # Create the env
    self.env = create_environment(env_name)

    # Extract observation size and number of actions from the environment
    obs_size = self.env.observation_space.shape
    n_actions = self.env.action_space.n

    # Initialize the Q-network with Noisy DQN
    self.q_net = DQN(hidden_size, obs_size, n_actions, sigma=sigma)

    # Initialize the target Q-network by copying the Q-network
    self.target_q_net = copy.deepcopy(self.q_net)

    # Set the action selection policy
    self.policy = policy

    # Initialize the experience replay buffer
    self.buffer = ReplayBuffer(capacity=capacity)

    self.save_hyperparameters()

   # Fill the experience buffer with samples
    while len(self.buffer) < self.hparams.samples_per_epoch:
      print(f"{len(self.buffer)} samples in experience buffer. Filling...")
      self.play_episode()

  @torch.no_grad()
  def play_episode(self, policy=None):
    """
    Simulate an episode and collect samples to fill the experience buffer.

    Parameters:
    - policy (function): Action selection policy (default: None).
    """
    state = self.env.reset()
    done = False

    while not done:
      if policy:
        action = policy(state, self.q_net)
      else:
        action = self.env.action_space.sample()

      next_state, reward, done, info = self.env.step(action)
      exp = (state, action, reward, done, next_state)
      self.buffer.append(exp)
      state = next_state

  # Forward.
  def forward(self, x):
    return self.q_net(x)

  # Configure optimizers.
  def configure_optimizers(self):
    q_net_optimizer = self.hparams.optim(self.q_net.parameters(), lr=self.hparams.lr)
    return [q_net_optimizer]

  # Create dataloader.
  def train_dataloader(self):
    dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch)
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=self.hparams.batch_size
    )
    return dataloader

  # Training step.
  def training_step(self, batch, batch_idx):
    """
    Execute a training step of a Deep Q Learning with Prioritized Experience Replay
    """
    # Unpack the components and make sure that is in the correct format
    indices, weights, states, actions, rewards, dones, next_states = batch
    weights = weights.unsqueeze(1)
    actions = actions.unsqueeze(1)
    rewards = rewards.unsqueeze(1)
    dones = dones.unsqueeze(1)

    # 1. Predict the [Q(s,a),Q(s,a),...] of the current state and actions
    #    by the Q-network.
    state_action_values = self.q_net(states).gather(1, actions)

    # 2. Calculate the target [Q(s,a),Q(s,a),...] (for comparation and updating)
    with torch.no_grad():
      # Predict the BEST actions for the next state by the Q-network
      _, next_actions = self.q_net(next_states).max(dim=1, keepdim=True)
      # By the Target-Q-network, evaluate the [Q(s',a*), Q(s',a*), ...]
      next_action_values = self.target_q_net(next_states).gather(1, next_actions)
      # Make sure you not include any extra rewards.
      next_action_values[dones] = 0.0

    # Compute the expected Q(s,a)
    expected_state_action_values = rewards + self.hparams.gamma * next_action_values

    # Compute the TD error between the actual Q(s,a) and the Target Q(s',a*)
    td_errors = (state_action_values - expected_state_action_values).abs().detach()

    # Update the priorities if the experiences, based on their TD.
    # -> the more TD, the more potential learning.
    for idx, e in zip(indices, td_errors):
      self.buffer.update(idx, e.item())

    # Compute the loss
    loss = weights * self.hparams.loss_fn(state_action_values, expected_state_action_values, reduction='none')
    loss = loss.mean()

    self.log('episode/Q-Error', loss)
    # Return the loss for back propegation (update the DQN weights.)
    return loss

  # Training epoch end.
  def training_epoch_end(self, training_step_outputs):
    """
    Callback function where the training epoch is over.
    """
    # Update the alpha & beta
    alpha = max(
        self.hparams.a_end,
        self.hparams.a_start - self.current_epoch / self.hparams.a_last_episode
    )
    beta = min(
        self.hparams.b_end,
        self.hparams.b_start + self.current_epoch / self.hparams.b_last_episode
    )
    # set the newer alpha & beta
    self.buffer.alpha = alpha
    self.buffer.beta = beta

    # Play one epicode for update the memory with more newer experiences.
    self.play_episode(policy=self.policy)
    self.log('episode/Return', self.env.unwrapped.game_state.score())

    # Update the Target netwoek weigths each `sync_rate` times.
    if self.current_epoch % self.hparams.sync_rate == 0:
      self.target_q_net.load_state_dict(self.q_net.state_dict())

#### Purge logs and run the visualization tool (Tensorboard)

In [ ]:
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

#### Train the policy

In [ ]:
# Create an instance of the agent
algo = DeepQLearning(
  'Catcher-v0',
  lr=0.0005,
  sigma=0.5,
  hidden_size=512,
  samples_per_epoch=1_000,
  a_last_episode=1_200,
  b_last_episode=1_200
)

# Create a trainer object
trainer = Trainer(
  gpus=num_gpus,
  max_epochs=1_500,
  log_every_n_steps=1
)

# training the agent
trainer.fit(algo)

#### Check the resulting policy

In [ ]:
env = algo.env
policy = algo.policy
q_net = algo.q_net.cuda()
frames = []

for episode in range(10):
  done = False
  obs = env.reset()
  while not done:
    frames.append(env.render(mode='rgb_array'))
    action = policy(obs, q_net)
    obs, _, done, _ = env.step(action)

In [ ]:
display_video(frames)